# RAG 

# Initialisation

In [1]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
	# Commandes à exécuter uniquement sur Google Colab
	!git clone https://github.com/Zbrlaa/Rag
	%cd Rag
	!pip install -r requirements.txt
else:
	print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Cloning into 'Rag'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 134 (delta 5), reused 4 (delta 3), pack-reused 121 (from 1)
Receiving objects: 100% (134/134), 94.65 MiB | 12.71 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/Rag
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 111.5 MB/s eta 

# Exercice 1 : Indexation

In [2]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("data/", use_multithreading=True)
documents = loader.load()
print(f"{len(documents)} documents")

65 documents


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
	chunk_size=1000,
	chunk_overlap=200,
	add_start_index=True,
	separators=["\n\n", "\n", " ", ""] #Ordre de priorité des caractères de découpe
)

all_splits = text_splitter.split_documents(documents)

print(f"Nombre de paragraphes : {len(all_splits)}")

Nombre de paragraphes : 8919


In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
	model_name="intfloat/multilingual-e5-base"
)

/tmp/ipython-input-464981128.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
	documents=all_splits, 
	embedding=embedding_model,
	persist_directory="./chroma_db"
)

print(f"Nombre d'éléments dans la base : {vectorstore._collection.count()}")

Nombre d'éléments dans la base : 8919


# Exercice 2 : Interrogation

In [6]:
def ask_base(query, k=3):
    # Recherche docs les plus proches et scores de distance
    docs_and_scores = vectorstore.similarity_search_with_score(query, k)
    
    return docs_and_scores

In [7]:
ma_query = "IA driving"

result = ask_base(ma_query, 2)

for doc, score in result:
	print(f"- Score de distance : {score:.4f}")
	print(f"- Source : {doc.metadata.get('source', 'Inconnue')}")
	print(f"- Start Index : {doc.metadata.get('start_index', 0)}")
	print("-" * 50)

- Score de distance : 0.3397
- Source : data/autres_articles/2412.18607v1.pdf
- Start Index : 8737
--------------------------------------------------
- Score de distance : 0.3576
- Source : data/autres_articles/2412.18604v1.pdf
- Start Index : 43319
--------------------------------------------------


# Exercice 3 : Prompt Template

In [8]:
from langchain_core.prompts import PromptTemplate

template = """Vous êtes un assistant spécialisé dans l'analyse de documents. 
Utilisez les éléments de contexte suivants pour répondre à la question posée. 
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE : 
{context}

QUESTION : 
{question}

RÉPONSE :"""

rag_prompt = PromptTemplate(
	template=template, 
	input_variables=["context", "question"]
)

print(rag_prompt.format(context="1", question="2"))

Vous êtes un assistant spécialisé dans l'analyse de documents. 
Utilisez les éléments de contexte suivants pour répondre à la question posée. 
Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.

CONTEXTE : 
1

QUESTION : 
2

RÉPONSE :


# Exercice 4 : chaîne RAG

In [ ]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

TypeError: str expected, not NoneType

In [9]:
def ask_llm_with_rag(question, k=10):
	docs_and_scores = ask_base(question, k=k)
	
	context = "\n\n".join([doc.page_content for doc, score in docs_and_scores])
	
	prompt_final = rag_prompt.format(context=context, question=question)

	response = llm.invoke(full_prompt)
	
	return response.content

ask_llm_with_rag("IA driving ?")

NameError: name 'llm' is not defined